In [1]:
!pip install bitsandbytes
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 32.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
import bitsandbytes as bnb
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from torch.utils.data import DataLoader, Dataset
import torch
from datasets import load_dataset
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import numpy as np
import os
from huggingface_hub import login

os.environ["HF_TOKEN"] = "HF_TOKEN_HERE"
login(token=os.environ["HF_TOKEN"])

2026-02-18 17:49:24.312069: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1771436964.498335      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1771436964.554490      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1771436964.998364      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771436964.998401      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1771436964.998404      24 computation_placer.cc:177] computation placer alr

# Dataset

In [3]:
cp = "neil-code/dialogsum-test"
ds = load_dataset(cp)

README.md: 0.00B [00:00, ?B/s]

train.csv: 0.00B [00:00, ?B/s]

validation.csv: 0.00B [00:00, ?B/s]

test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1999 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/499 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/499 [00:00<?, ? examples/s]

In [4]:
def add_instruction(dct):
    return {
        "dialouge_with_sum": f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

{dct['dialogue']}

### Output:
{dct['summary']} <|endoftext|>""",
        "dialouge_without_sum": f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

{dct['dialogue']}

### Output:
""",
    }

In [5]:
ds = ds.map(add_instruction)

Map:   0%|          | 0/1999 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

Map:   0%|          | 0/499 [00:00<?, ? examples/s]

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'dialouge_with_sum', 'dialouge_without_sum'],
        num_rows: 1999
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'dialouge_with_sum', 'dialouge_without_sum'],
        num_rows: 499
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'dialouge_with_sum', 'dialouge_without_sum'],
        num_rows: 499
    })
})

In [7]:
ds = ds.remove_columns(["dialogue", "id", "topic", "summary"])

In [8]:
train_dataset = ds["train"]
val_dataset = ds["validation"]

In [9]:
train_dataset

Dataset({
    features: ['dialouge_with_sum', 'dialouge_without_sum'],
    num_rows: 1999
})

In [10]:
print(ds["train"][0]["dialouge_with_sum"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [11]:
print(ds["train"][0]["dialouge_without_sum"])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [13]:
cp = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(cp)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [14]:
tokenizer.pad_token = tokenizer.eos_token

In [15]:
tokenizer.eos_token

'<|endoftext|>'

In [16]:
train_dataset

Dataset({
    features: ['dialouge_with_sum', 'dialouge_without_sum'],
    num_rows: 1999
})

In [17]:
def customCollate(batch):
    texts = [dct["dialouge_with_sum"] for dct in batch]
    questions = [dct["dialouge_without_sum"] for dct in batch]
    texts = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    mask_ids = torch.ones_like(texts["input_ids"])
    # marking question with label -100
    labels = {
        "input_ids": texts["input_ids"].clone(),
        "attention_mask": texts["attention_mask"].clone(),
    }
    for i in range(len(questions)):
        question = questions[i]
        question = tokenizer(question, return_tensors="pt", padding=False)
        question_len = len(question["input_ids"][0])
        labels["input_ids"][i][:question_len] = -100
    labels["input_ids"][labels["input_ids"] == tokenizer.pad_token_id] = -100
    texts = {k: v.to(device) for k, v in texts.items()}
    return {"texts": texts, "labels": labels["input_ids"].to(device)}

In [18]:
batch_size = 1
train_ds = DataLoader(train_dataset, batch_size=batch_size, collate_fn=customCollate)
val_ds = DataLoader(val_dataset, batch_size=batch_size, collate_fn=customCollate)

In [19]:
batch1 = next(iter(train_ds))

In [20]:
batch1

{'texts': {'input_ids': tensor([[21106,   318,   281, 12064,   326,  8477,   257,  4876,    11, 20312,
             351,   281,  5128,   326,  3769,  2252,  4732,    13, 19430,   257,
            2882,   326, 20431, 32543,   262,  2581,    13,   198, 50284,   198,
           21017, 20689,    25,  5060,  3876,   786,   262,  1708, 42427,    13,
             198,   198,     2, 15439,    16,     2,    25, 15902,    11,  1770,
              13,  4176,    13,   314,  1101,  9356, 32534,    13,  4162,   389,
             345,   994,  1909,    30,   198,     2, 15439,    17,     2,    25,
             314,  1043,   340,   561,   307,   257,   922,  2126,   284,   651,
             257,  2198,    12,   929,    13,   198,     2, 15439,    16,     2,
              25,  3363,    11,   880,    11,   345,  4398,   470,   550,   530,
             329,   642,   812,    13,   921,   815,   423,   530,   790,   614,
              13,   198,     2, 15439,    17,     2,    25,   314,   760,    13,
      

# Model

In [21]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    cp, quantization_config=quantization_config, device_map="auto"
)

config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [22]:
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable()

In [23]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "o_proj", "dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, peft_config=lora_config)

In [24]:
total_trainable_params = 0
total_params = 0
for params in model.parameters():
    total_params += params.numel()
    if params.requires_grad:
        total_trainable_params += params.numel()
total_trainable_params * 100 / total_params

1.0232530252915373

In [25]:
with torch.no_grad():
    out = model(**batch1["texts"], labels=batch1["labels"])

In [26]:
out.loss.item()

2.1140952110290527

In [27]:
torch.cuda.empty_cache()

In [28]:
optimizer = AdamW([i for i in model.parameters() if i.requires_grad], lr=5e-5)

In [29]:
n_epochs = 2
total_steps = len(train_ds) * n_epochs
warmup_steps = int(0.1 * total_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
)

In [30]:
def train_epoch(model, train_ds, optimizer, scheduler, accumulation_step=2):
    model.train()
    tot_loss = 0
    step_comp = 0
    i = 0
    progress = tqdm(train_ds, desc="Training")
    for batch in progress:
        try:
            i += 1
            out = model(**batch["texts"], labels=batch["labels"])
            loss = out.loss
            loss.backward()

            tot_loss += loss.item()
            progress.set_postfix({"loss": f"{loss.item():.4f}"})
            if (i) % accumulation_step == 0 or i == len(train_ds) - 1:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
        except Exception as E:
            print(f"Error at batch {i}: {E}")
            torch.cuda.empty_cache()
            optimizer.zero_grad()

    return tot_loss / len(train_ds)

In [31]:
def val(model, val_ds):
    model.eval()
    progress = tqdm(val_ds, desc="Validating")
    total_loss = 0
    num_batches = 0

    for batch in progress:
        try:
            with torch.no_grad():
                out = model(**batch["texts"], labels=batch["labels"])
                if not np.isnan(out.loss.item()):
                    total_loss += out.loss.item()
                    num_batches += 1
                progress.set_postfix({"loss": f"{out.loss.item():.4f}"})
        except Exception as e:
            print(f"Error processing batch: {e}")
            torch.cuda.empty_cache()
            continue

    avg_loss = total_loss / num_batches if num_batches > 0 else float("inf")
    return avg_loss

In [32]:
len(val_ds)

499

In [33]:
len(train_ds)

1999

In [34]:
%%time
# model output before training
batch1 = next(iter(train_ds))
input1 = (
    tokenizer.batch_decode(batch1["texts"]["input_ids"])[0].split("### Output")[0]
    + "### Output:\n"
)
input1 = tokenizer(input1, return_tensors="pt")
input1 = {k: v.to(device) for k, v in input1.items()}
out = model.generate(**input1)
print(tokenizer.batch_decode(out)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [35]:
lss = val(model, val_ds)

print(f"(perplexity before training :", f"{np.exp(lss):.4f}")

Validating: 100%|██████████| 499/499 [03:49<00:00,  2.18it/s, loss=1.4358]

(perplexity before training : 4.9777


In [36]:
model.train()
for epoch in range(1, n_epochs + 1):
    print(f"Epoch {epoch}")
    train_loss = train_epoch(model, train_ds, optimizer, scheduler)
    print(f"loss in epoch {epoch} :", f"{train_loss:.4f}, perplexity : {np.exp(train_loss)}")
    lss = val(model, val_ds)
    print(f"(val loss after epoch {epoch} :", f"{lss:.4f} perplexity : {np.exp(lss)}")

Epoch 1


Training: 100%|██████████| 1999/1999 [45:57<00:00,  1.38s/it, loss=0.3482]


loss in epoch 1 : 1.1049, perplexity : 3.0190658691891175


Validating: 100%|██████████| 499/499 [03:49<00:00,  2.17it/s, loss=0.7577]


(val loss after epoch 1 : 0.9510 perplexity : 2.5882938993626134
Epoch 2


Training: 100%|██████████| 1999/1999 [45:58<00:00,  1.38s/it, loss=0.2521]


loss in epoch 2 : 0.9487, perplexity : 2.582308628879826


Validating: 100%|██████████| 499/499 [03:49<00:00,  2.17it/s, loss=0.6911]

(val loss after epoch 2 : 0.9252 perplexity : 2.5224491851870803


In [37]:
%%time
# model output after training
batch1 = next(iter(train_ds))
input1 = (
    tokenizer.batch_decode(batch1["texts"]["input_ids"])[0].split("### Output")[0]
    + "### Output:\n"
)
input1 = tokenizer(input1, return_tensors="pt")
input1 = {k: v.to(device) for k, v in input1.items()}
out = model.generate(**input1)
print(tokenizer.batch_decode(out)[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    
### Instruct: Summarise the following Conversation.

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#P

In [38]:
!mkdir /kaggle/working/phi-2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [39]:
model.save_pretrained("/kaggle/working/phi-2")

In [40]:
from huggingface_hub import login, upload_folder

# (optional) Login with your Hugging Face credentials
login()

# Push your model files
upload_folder(
    folder_path="/kaggle/working/phi-2", repo_id="Pankaj121212/phi-2-cus", repo_type="model"
)

Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/Pankaj121212/phi-2-cus/commit/34982d83275006e7a25da528120da0ed9e08b9c6', commit_message='Upload folder using huggingface_hub', commit_description='', oid='34982d83275006e7a25da528120da0ed9e08b9c6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Pankaj121212/phi-2-cus', endpoint='https://huggingface.co', repo_type='model', repo_id='Pankaj121212/phi-2-cus'), pr_revision=None, pr_num=None)